In [1]:
import lyricsgenius
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from flair.models import TextClassifier
from flair.data import Sentence
from google.cloud import language
from google.cloud.language import enums, types
classifier = TextClassifier.load("sentiment")

2020-07-31 16:45:23,294 loading file C:\Users\LuisLGM\.flair\models\sentiment-en-mix-distillbert.pt


In [2]:
## Necesario Client Token de Genius obtener en https://docs.genius.com/
genius = lyricsgenius.Genius("")

# Analisis sentimiento cada estrofa

## 1. Usando Flair

In [22]:
def analisis_estrofa_flair (f):
    f = re.sub(r'[\(\[].*?]', '', f)
    f = re.sub(r'\n\n\n', '', f)
    percents_positives = 0
    percents_positives1 = 0
    num_sentence = 0

    for sentence in f.split('\n\n'):
        sent_predict = Sentence (sentence)
        classifier.predict(sent_predict)
        if sent_predict.labels[0].value == "POSITIVE":
            percents_positives += sent_predict.labels[0].score
        else :
            percents_positives1 += (1 - sent_predict.labels[0].score)
        num_sentence += 1

    percents_positives += percents_positives1
    percent_positive = (percents_positives/float(num_sentence))*100
    percent_negative = 100 - percent_positive

    print ("percent positive: ", percent_positive)
    print ("percent negative: ", percent_negative)


### Canción de Rap

In [25]:
song = genius.search_song("Nonstop",artist="Drake")
f = song.lyrics
print(f)

Searching for "Nonstop" by Drake...
Done.
[Intro: Lil Juice]
Tay Keith, fuck these niggas up!

[Verse 1: Drake]
Look, I just flipped a switch (Flipped, flipped)
I don't know nobody else that's doin' this
Bodies start to drop, ayy (Hit the floor)
Now they wanna know me since I hit the top, ayy
This a Rollie, not a stopwatch, shit don't ever stop
This the flow that got the block hot, shit got super hot, ayy
Give me my respect (Give me my respect)
I just took it left like I'm ambidex'
Bitch, I move through London with the Eurostep (Two)
Got a sneaker deal and I ain't break a sweat
Catch me 'cause I'm gone (Outta there, I’m gone)
How I go from 6 to 23 like I'm LeBron?
Servin' up a pack (Servin' up a pack)
Niggas pullin' gimmicks 'cause they scared to rap, ayy
Funny how they shook, ayy, got these niggas shook
Pullin' back the curtain by myself, take a look, ayy
I'm a bar spitta, I’m a hard hitta
Yeah I’m light-skinned, but I'm still a dark nigga
I'm a wig splitta, I'm a tall figure
I'm a un

In [26]:
analisis_estrofa_flair(f)

percent positive:  4.24018899599711
percent negative:  95.75981100400288


### Canción de Pop

In [29]:
song = genius.search_song("Out Of the Woods",artist="Taylor Swift")
f = song.lyrics
print(f)

Searching for "Out Of the Woods" by Taylor Swift...
Done.
[Verse 1]
Looking at it now
It all seems so simple
We were lying on your couch
I remember
You took a Polaroid of us
Then discovered (Then discovered)
The rest of the world was black and white
But we were in screaming color
And I remember thinking

[Chorus]
Are we out of the woods yet? Are we out of the woods yet?
Are we out of the woods yet? Are we out of the woods?
Are we in the clear yet? Are we in the clear yet?
Are we in the clear yet? In the clear yet? Good
Are we out of the woods yet? Are we out of the woods yet?
Are we out of the woods yet? Are we out of the woods?
Are we in the clear yet? Are we in the clear yet?
Are we in the clear yet? In the clear yet? Good
(Are we out of the woods?)

[Verse 2]
Looking at it now
Last December (Last December)
We were built to fall apart
Then fall back together (Back together)
Your necklace hanging from my neck
The night we couldn't quite forget
When we decided, we decided
To move the f

In [30]:
analisis_estrofa_flair(f)

percent positive:  68.15170049667358
percent negative:  31.848299503326416


### Canción de Country

In [31]:
song = genius.search_song("heaven",artist="kane brown")
f = song.lyrics
print(f)

Searching for "heaven" by kane brown...
Done.
[Verse 1]
This is perfect
Come kiss me one more time
I couldn't dream this up
Even if I tried
You and me in this moment
Feels like magic, don't it?
I'm right where I wanna be

[Chorus]
Everybody's talkin' about heaven like they just can't wait to go
Sayin' how it's gonna be so good, so beautiful
Lyin' next to you, in this bed with you, I ain't convinced
'Cause, I don't know how, I don't know how heaven, heaven
Could be better than this

[Verse 2]
I swear you're an angel
Sent to this world
What did I do right to deserve you, girl?
I could stay here forever
I'd be fine if we never
Had to even leave this room

[Chorus]
Everybody's talkin' 'bout heaven like they just can't wait to go
Sayin' how it's gonna be so good, so beautiful
Lyin' next to you, in this bed with you, I ain't convinced
'Cause, I don't know how, I don't know how heaven, heaven
Could be better than this
Could be better than this
[Chorus]
Everybody's talkin' 'bout heaven like th

In [32]:
analisis_estrofa_flair(f)

percent positive:  99.93821680545807
percent negative:  0.06178319454193115


### Canción Alternativo

In [33]:
song = genius.search_song("Rylan",artist="The national")
f = song.lyrics
print(f)

Searching for "Rylan" by The national...
Done.
[Verse 1]
Rylan, you should try to get some sun
You remind me of everyone
Rylan, did you break your mother's heart?
Every time you tried to play your part

[Chorus]
Is it easy to keep so quiet?
Everybody loves a quiet child
Underwater, you're almost free
If you wanna be alone, come with me

[Verse 2]
Rylan, we can take the quick way out
We can turn blank white in a blank white house
Say that you're a pervert, you're a vulture
Don't you wanna be popular culture?

[Chorus]
Is it easy to keep so quiet?
Everybody loves a quiet child
Underwater, you're almost free
If you wanna be alone, come with me
Is it easy to live inside yourself?
All the little kids are high and hazy
Everybody's got nowhere to go
Everybody wants to be amazing

[Bridge]
Rylan, California's rotten
Dress light blue to be forgotten
Eat your pearls on Sunday morning
Keep your conversations boring
Stay with me among the strangers
Change your mind and nothing changes
Don't let sh

In [34]:
analisis_estrofa_flair(f)

percent positive:  0.08368492126464844
percent negative:  99.91631507873535


## 2. Usando Vader

In [49]:
def analisis_estrofa_vader (f):
    sid = SentimentIntensityAnalyzer()
    f = re.sub(r'[\(\[].*?]', '', f)
    # f = re.sub(r'\n\n', '', f)

    num_positive = 0
    num_negative = 0
    num_neutral = 0
    num_total = 0
    percent_negative = 0
    percent_neutral = 0
    percent_positive = 0

    for sentence in f.split('\n\n'):
        comp = sid.polarity_scores(sentence)
        comp = comp['compound']
        if comp >= 0.5:
            num_positive += 1
        elif comp > -0.5 and comp < 0.5:
            num_neutral += 1
        else:
            num_negative += 1

    num_total = num_negative + num_neutral + num_positive
    percent_negative = (num_negative/float(num_total))*100
    percent_neutral = (num_neutral/float(num_total))*100
    percent_positive = (num_positive/float(num_total))*100

    print ("Negative: ", percent_negative)
    print ("Neutral: ", percent_neutral)
    print ("Positive: ", percent_positive)

### Canción de Rap

In [103]:
song = genius.search_song("Nonstop",artist="Drake")
f = song.lyrics

Searching for "Nonstop" by Drake...
Done.


In [104]:
analisis_estrofa_vader(f)

Negative:  20.0
Neutral:  40.0
Positive:  40.0


### Canción de Pop

In [105]:
song = genius.search_song("Out Of the Woods",artist="Taylor Swift")
f = song.lyrics

Searching for "Out Of the Woods" by Taylor Swift...
Done.


In [106]:
analisis_estrofa_vader(f)

Negative:  16.666666666666664
Neutral:  0.0
Positive:  83.33333333333334


### Canción de Country

In [107]:
song = genius.search_song("heaven",artist="kane brown")
f = song.lyrics

Searching for "heaven" by kane brown...
Done.


In [108]:
analisis_estrofa_vader(f)

Negative:  0.0
Neutral:  16.666666666666664
Positive:  83.33333333333334


### Canción Alternativo

In [109]:
song = genius.search_song("Rylan",artist="The national")
f = song.lyrics

Searching for "Rylan" by The national...
Done.


In [110]:
analisis_estrofa_vader(f)

Negative:  50.0
Neutral:  16.666666666666664
Positive:  33.33333333333333


## 3. Usando GCloud

In [111]:
def analyze_text_sentiment(text):
    client = language.LanguageServiceClient()
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    response = client.analyze_sentiment(document=document)

    sentiment = response.document_sentiment
    results = [
        #('text', text),
        sentiment.score,
        sentiment.magnitude
    ]
    return results

In [59]:
def analisis_estrofa_g_cloud(f):
    results = 0
    cont = 0
    for sentence in f.split('\n\n'):
        result = analyze_text_sentiment (f)
        results += result[0]
        cont +=1
    analysis = results/cont
    print (analysis)

### Canción de rap

In [112]:
song = genius.search_song("Nonstop",artist="Drake")
f = song.lyrics

Searching for "Nonstop" by Drake...
Done.


In [61]:
analisis_estrofa_g_cloud(f)

-0.4000000059604645


### Canción de pop

In [113]:
song = genius.search_song("Out Of the Woods",artist="Taylor Swift")
f = song.lyrics

Searching for "Out Of the Woods" by Taylor Swift...
Done.


In [63]:
analisis_estrofa_g_cloud(f)

0.0


### Canción de Country

In [114]:
song = genius.search_song("heaven",artist="kane brown")
f = song.lyrics

Searching for "heaven" by kane brown...
Done.


In [65]:
analisis_estrofa_g_cloud(f)

0.5


### Canción Alternativo

In [115]:
song = genius.search_song("Rylan",artist="The national")
f = song.lyrics

Searching for "Rylan" by The national...
Done.


In [67]:
analisis_estrofa_g_cloud(f)

-0.10000000149011612


# Analisis Sentimiento cada verso

## 1. Usando Flair

In [68]:
def analisis_verso_flair (f):
    f = re.sub(r'[\(\[].*?]', '', f)
    f = re.sub(r'\n\n', '', f)
    f = f[1:]
    percents_positives = 0
    percents_positives1 = 0
    num_sentence = 0

    for sentence in f.split('\n'):
        sent_predict = Sentence (sentence)
        classifier.predict(sent_predict)
        if sent_predict.labels[0].value == "POSITIVE":
            percents_positives += sent_predict.labels[0].score
        else :
            percents_positives1 += (1 - sent_predict.labels[0].score)
        num_sentence += 1

    percents_positives += percents_positives1
    percent_positive = (percents_positives/float(num_sentence))*100
    percent_negative = 100 - percent_positive

    print ("percent positive: ", percent_positive)
    print ("percent negative: ", percent_negative)

#### Canción de rap

In [69]:
song = genius.search_song("Nonstop",artist="Drake")
f =song.lyrics

Searching for "Nonstop" by Drake...
Done.


In [70]:
analisis_verso_flair (f)

percent positive:  66.66695261001587
percent negative:  33.33304738998413


### Canción de pop

In [72]:
song = genius.search_song("Out Of the Woods",artist="Taylor Swift")
f = song.lyrics

Searching for "Out Of the Woods" by Taylor Swift...
Done.


In [73]:
analisis_verso_flair (f)

percent positive:  71.63045618632069
percent negative:  28.369543813679314


### Canción de Country

In [74]:
song = genius.search_song("heaven",artist="kane brown")
f = song.lyrics

Searching for "heaven" by kane brown...
Done.


In [75]:
analisis_verso_flair (f)

percent positive:  70.78294734801015
percent negative:  29.217052651989846


### Canción Alternativo

In [76]:
song = genius.search_song("Rylan",artist="The national")
f = song.lyrics

Searching for "Rylan" by The national...
Done.


In [77]:
analisis_verso_flair (f)

percent positive:  60.652191323392536
percent negative:  39.347808676607464


## 2. Usando Vader

In [81]:
def analisis_verso_vader (f):
    sid = SentimentIntensityAnalyzer()
    f = re.sub(r'[\(\[].*?]', '', f)
    f = re.sub(r'\n\n', '', f)

    num_positive = 0
    num_negative = 0
    num_neutral = 0

    for sentence in f.split('\n'):
        comp = sid.polarity_scores(sentence)
        comp = comp['compound']
        if comp >= 0.5:
            num_positive += 1
        elif comp > -0.5 and comp < 0.5:
            num_neutral += 1
        else:
            num_negative += 1

    num_total = num_negative + num_neutral + num_positive
    percent_negative = (num_negative/float(num_total))*100
    percent_neutral = (num_neutral/float(num_total))*100
    percent_positive = (num_positive/float(num_total))*100

    print ("Negative: ", percent_negative)
    print ("Neutral: ", percent_neutral)
    print ("Positive: ", percent_positive)

### Canción de rap

In [82]:
song = genius.search_song("Nonstop",artist="Drake")
f =song.lyrics

Searching for "Nonstop" by Drake...
Done.


In [83]:
analisis_verso_vader(f)

Negative:  12.162162162162163
Neutral:  81.08108108108108
Positive:  6.756756756756757


### Canción de pop

In [84]:
song = genius.search_song("Out Of the Woods",artist="Taylor Swift")
f = song.lyrics

Searching for "Out Of the Woods" by Taylor Swift...
Done.


In [85]:
analisis_verso_vader(f)

Negative:  1.3888888888888888
Neutral:  72.22222222222221
Positive:  26.38888888888889


### Canción de Country

In [86]:
song = genius.search_song("heaven",artist="kane brown")
f = song.lyrics

Searching for "heaven" by kane brown...
Done.


In [87]:
analisis_verso_vader (f)

Negative:  9.375
Neutral:  65.625
Positive:  25.0


### Canción Alternativo

In [88]:
song = genius.search_song("Rylan",artist="The national")
f = song.lyrics

Searching for "Rylan" by The national...
Done.


In [89]:
analisis_verso_vader (f)

Negative:  11.428571428571429
Neutral:  82.85714285714286
Positive:  5.714285714285714


## 3. Usando GCloud

In [90]:
def analyze_text_sentiment(text):
    client = language.LanguageServiceClient()
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    response = client.analyze_sentiment(document=document)

    sentiment = response.document_sentiment
    results = [
        #('text', text),
         sentiment.score,
        sentiment.magnitude
        ]
    return results

In [92]:
def analisis_verso_g_cloud(f):
    results = 0
    cont = 0
    for sentence in f.split('\n'):
        result = analyze_text_sentiment (f)
        results += result[0]
        cont +=1
    analysis = results/cont
    print (analysis)

### Canción de rap

In [93]:
song = genius.search_song("Nonstop",artist="Drake")
f =song.lyrics

Searching for "Nonstop" by Drake...
Done.


In [94]:
analisis_verso_g_cloud(f)

-0.4000000059604645


### Canción de pop

In [96]:
song = genius.search_song("Out Of the Woods",artist="Taylor Swift")
f = song.lyrics

Searching for "Out Of the Woods" by Taylor Swift...
Done.


In [97]:
analisis_verso_g_cloud(f)

0.0


### Canción de Country

In [99]:
song = genius.search_song("heaven",artist="kane brown")
f = song.lyrics

Searching for "heaven" by kane brown...
Done.


In [100]:
analisis_verso_g_cloud(f)

0.5


### Canción Alternativo

In [101]:
song = genius.search_song("Rylan",artist="The national")
f = song.lyrics

Searching for "Rylan" by The national...
Done.


In [102]:
analisis_verso_g_cloud(f)

-0.10000000149011612
